# Image Thresholding

[Thresholding (image processing) - Wikipedia](https://en.wikipedia.org/wiki/Thresholding_(image_processing))

[Image Thresholding (OpenCV documentation)](https://docs.opencv.org/master/d7/d4d/tutorial_py_thresholding.html)

[Basic Thresholding Operations](https://docs.opencv.org/3.4/db/d8e/tutorial_threshold.html)



If we convert a *color image* into a *grayscale* one, we'll see shapes, edges and also changes in the grey color intensity (shades of grey). If we're not interested in grey intensity but only object shapes and edges, we can go one step further and convert grey image into a *binary* one where pixels are either black or white. This is called binary thresholding.

In [ ]:
import cv2
import matplotlib.pyplot as plt
img = cv2.imread('../data/rainbow.jpg')
print(f'type(img) = {type(img)}, img.shape = {img.shape}')
plt.imshow(img)

In [ ]:
img = cv2.imread('../data/rainbow.jpg', 0)
plt.imshow(img)

In [ ]:
plt.imshow(img, cmap='gray')

[retval, dst = cv.threshold(src, thresh, maxval, type[, dst])
](https://docs.opencv.org/master/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57)

*  The function is typically used to get a bi-level (binary) image out of a grayscale image

Arguments:
* src - input array (input image)
* thresh - threshold value; grayscale value of the pixel is compared to threshold
* maxval - maximum value to use with the THRESH_BINARY and THRESH_BINARY_INV thresholding types; this will be the new value of the pixel in the output image if current greyscale value is higher(or lower) than threshold
* type - thresholding type (see [ThresholdTypes](https://docs.opencv.org/master/d7/d1b/group__imgproc__misc.html#gaa9e58d2860d4afa658ef70a9b1115576))


* THRESH_BINARY: every value below threshold should turn into 0 (black) and every value above the threshold should turn to maxval (white)
* THRESH_BINARY_INV: every value below threshold should turn into maxval (white) and every value above the threshold should turn to 0 (black)

Return value: the computed threshold value if Otsu's or Triangle methods used.

Typically, threshold value is chosen to be the value between the minimum (0) and maximum. To find the maximum value we can use function [ndarray.max(axis=None, out=None, keepdims=False, initial=<no value>, where=True)](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.max.html) which returns the maximum along a given axis.


In [ ]:
img.max()

In [ ]:
threshold_value = img.max() // 2 # floor division to get the whole integer
print(f'threshold_value = {threshold_value}')

# THRESH_BINARY: every value below threshold should turn into 0 (black) and every value above the threshold should turn to maxval (white)
ret, thresh1 = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)
print(ret)
plt.imshow(thresh1, cmap='gray')

In [ ]:
# THRESH_BINARY_INV: every value below threshold should turn into maxval (white) and every value above the threshold should turn to 0 (black)
ret, thresh1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
print(ret)
plt.imshow(thresh1, cmap='gray')

In [ ]:
# THRESH_TRUNC: if grayscale value is > threshold, turn it to maxvalue, otherwise keep it as it is
ret, thresh1 = cv2.threshold(img, 127, 255, cv2.THRESH_TRUNC)
print(ret)
plt.imshow(thresh1, cmap='gray')

In [ ]:
# THRESH_TOZERO: if grayscale value is < threshold, turn it to zero (black), otherwise keep it as it is
ret, thresh1 = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO)
print(ret)
plt.imshow(thresh1, cmap='gray')

In [ ]:
# THRESH_TOZERO_INV: if grayscale value is > threshold, turn it to zero (black), otherwise keep it as it is
ret, thresh1 = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO_INV)
print(ret)
plt.imshow(thresh1, cmap='gray')

In [ ]:
img = cv2.imread('../data/crossword.jpg', 0)
print(f'type(img) = {type(img)}, img.shape = {img.shape}')
plt.imshow(img, cmap='gray')

In [ ]:
def show_pic(img):
    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap='gray')

In [ ]:
show_pic(img)

 We're interested only in binary image.

In [ ]:
ret, thresh1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
print(ret)
show_pic(thresh1)

Some of the grey pixels got thresholded to white but we wanted them to be black.
We can try to use different thresholding types or change a threshold. If we increase threshold value, the darker gray pixels will be converted to black.

In [ ]:
ret, thresh1 = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)
print(ret)
show_pic(thresh1)

In [ ]:
ret, thresh1 = cv2.threshold(img, 180, 255, cv2.THRESH_BINARY)
print(ret)
show_pic(thresh1)

## Adaptive thresholding

[dst = cv.adaptiveThreshold(src, maxValue, adaptiveMethod, thresholdType, blockSize, C[, dst])
](https://docs.opencv.org/master/d7/d1b/group__imgproc__misc.html#ga72b913f352e4a1b1b397736707afcde3)

`cv.threshold()` function might not give desired result because the input grayscale image might have different parts with different values for foreground object and its background. Imagine we have an image with two parts, one has foreground object, let's say a letter A with grayscale value 80 and its background 40. The other part has foreground object, let's say a letter B with value 200 and background 100. If we apply threshold 127, simple BINARY threshold would turn 1st part of the image black and second to white. We can see here that these two parts of the image can't share the same threshold. There should be an algorithm that adapts the threshold for each pixel depending on its surrounding pixels, on the context in which it stands. 

* `src` - Source 8-bit single-channel image.
* `dst` - Destination image of the same size and the same type as src.
* `maxValue` - Non-zero value assigned to the pixels for which the condition is satisfied
* `adaptiveMethod` - Adaptive thresholding algorithm to use, see AdaptiveThresholdTypes. The BORDER_REPLICATE | BORDER_ISOLATED is used to process boundaries.
* `thresholdType` - Thresholding type that must be either THRESH_BINARY or THRESH_BINARY_INV, see ThresholdTypes.
* `blockSize` - Size of a pixel neighborhood that is used to calculate a threshold value for the pixel: 3, 5, 7, and so on.
* `C` - Constant subtracted from the mean or weighted mean (see the details below). Normally, it is positive but may be zero or negative as well.


* `THRESH_BINARY`: if pixel grayscale value is > T(x, y) (which is adaptive threshold value) then pixel turns to maxvalue, otherwise 0
* `THRESH_BINARY_INV`: if pixel grayscale value is > T(x, y) then pixel turns to 0, otherwise maxvalue

T(x, y) is a threshold calculated individually for each pixel. The algorithm selelected via `adaptiveMethod` parameter defines how this is done:

* `ADAPTIVE_THRESH_MEAN_C`: the threshold value T(x,y) is a mean of the blockSize×blockSize neighborhood of (x,y) minus C
* `ADAPTIVE_THRESH_GAUSSIAN_C`: the threshold value T(x,y) is a weighted sum (cross-correlation with a Gaussian window) of the blockSize×blockSize neighborhood of (x,y) minus C . The default sigma (standard deviation) is used for the specified blockSize .

In [ ]:
# values for blockSize and C are usually found after couple of iterations; we set some value and check if results is good enough
thresh2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 8)
show_pic(thresh2)

In [ ]:
blended = cv2.addWeighted(src1 = thresh1, alpha = 0.6, src2 = thresh2, beta=0.4, gamma=0)
show_pic(blended)

In [ ]:
thresh2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 8)
show_pic(thresh2)

In [ ]:
# sometimes satisfactory results can come out after we blend two threshold images
blended = cv2.addWeighted(src1 = thresh1, alpha = 0.6, src2 = thresh2, beta=0.4, gamma=0)
show_pic(blended)